
# **Understanding Embeddings with BERT**

In this notebook, we go into the world of embeddings, utilizing the BERT model to understand the semantic relationships between words in different contexts.

##  Setup and Installation

Start by installing the necessary libraries to ensure all functionalities are available.

In [ ]:
!pip install transformers==4.29.2
!pip install scipy==1.7.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.0
    Uninstalling transformers-4.41.0:
      Successfully uninstalled transformers-4.41.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 53.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
ERROR: pip's dependency reso

##  Importing Libraries

Import essential modules for our tasks.

In [ ]:
from transformers import BertModel, AutoTokenizer
from scipy.spatial.distance import cosine

/usr/local/lib/python3.10/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


##  Model Setup

Load the pre-trained BERT model and tokenizer. This model will help us extract embeddings for our analysis.

In [ ]:
# Defining the model name
model_name = "bert-base-cased"

# Loading the pre-trained model and tokenizer
model = BertModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


KeyboardInterrupt: 


##  Function Definition: Predict

Define a function that encodes input text into tensors, which are then fed to the model to obtain embeddings.

In [ ]:
# Defining a function to encode the input text and get model predictions
def predict(text):
    encoded_inputs = tokenizer(text, return_tensors="pt")
    # same as return model(**encoded_inputs)[0]
    return model(**encoded_inputs).last_hidden_state
    # The ** operator unpacks the encoded_inputs dictionary so that each key-value pair is passed as a separate argument to the model.
    # The model expects input in the form of keyword arguments like input_ids and attention_mask.

##  Defining the Sentences

Set up sentences to analyze. The

In [ ]:
# Defining the sentences
sentence3 = "The course is made for people who want to learn DSA from A to Z for free in a well-organized and structured manner."
sentence4 = "The course quality is better than what you get in paid courses, the only thing we don’t provide is doubt support, but trust me our YouTube video comments resolve that as well, we have a wonderful community of 250K+ people who engage in all of the videos."
sentence1 = "There was a fly drinking from my soup"
sentence2 = "There is a fly swimming in my juice"
# sentence2 = "To become a commercial pilot, he had to fly for 1500 hours." # second fly example

# Tokenizing the sentences
tokens1 = tokenizer.tokenize(sentence1)
tokens2 = tokenizer.tokenize(sentence2)

##  Tokenization and Model Predictions

Tokenize the sentences and obtain predictions (embeddings) from the model.

In [ ]:
# Getting model predictions for the sentences
out1 = predict(sentence1)
out2 = predict(sentence2)

# out1 is a tensor with shape [1, 8, 768]
# (assuming the hidden size is 768 and there are 8 tokens).
out1

### Extracting Embeddings with Technical Details

The line `emb1 = out1[0, tokens1.index("fly"), :].detach()` is extracting the embedding (hidden state representation) of the word "fly" from the BERT model's output for a given sentence. Here’s a detailed breakdown of each part:

#### Model Output: `out1`

- `out1 = predict(sentence1)`
- The `predict` function takes a sentence as input, tokenizes it, and passes it through the BERT model.
- The model’s output, `out1`, is a tensor containing the hidden states for each token in the input sentence. The shape of this tensor is `[batch_size, sequence_length, hidden_size]`.

#### Batch Dimension: `[0, ...]`

- `out1[0, ...]`
- Since the batch size is 1 (we're processing one sentence at a time), `out1[0, ...]` selects the first (and only) batch.

#### Token Index: `tokens1.index("fly")`

- `tokens1 = tokenizer.tokenize(sentence1)`
- This tokenizes the input sentence into individual tokens (subwords in the BERT vocabulary).
- `tokens1.index("fly")` finds the index of the token "fly" in the list of tokens.
- This index is used to access the hidden state corresponding to the token "fly".

#### Hidden State Dimension: `[..., :]`

- `out1[0, tokens1.index("fly"), :]`
- The `:` operator is used to select all hidden units (the hidden size dimension) for the token "fly".



In [ ]:
# Extracting embeddings for the word 'fly' in both sentences
emb1 = out1[0:, tokens1.index("fly"), :].detach()[0]
emb2 = out2[0:, tokens2.index("fly"), :].detach()[0]

# emb1 = out1[0:, 3, :].detach()
# emb2 = out2[0:, 3, :].detach()
emb1.size()

In [ ]:
emb1 = out1[0:, tokens1.index("fly"), :].detach()
emb1.size()

##  Calculating Cosine Similarity

Calculate the cosine similarity between the embeddings of the word "fly" from both sentences to measure how context affects meaning.

In [ ]:
# Calculating the cosine similarity between the embeddings
cosine(emb1, emb2)